In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import copy
from scipy import ndimage
from PIL import Image
import os

In [2]:
#setting the path for the folder to read ... 
path1 = 'F:\code files\machine learning\dogs vs cats classification\dataset'

#setting important values 
height = 64
width = 64

In [3]:
#function to load images from directory into array
def create_dataset(directory):
    img_array = []
    class_name = []
    m = 0
    
    for dir1 in os.listdir(directory):
        #print(dir1)
        for file in os.listdir(os.path.join(directory,dir1)):
            image_path = os.path.join(directory, dir1, file)
            try:
                image = np.array(Image.open(image_path))
                image = np.resize(image, (height,width,3))
                image = image.astype('float32')
                image /= 255
                img_array.append(image)
                if dir1 == 'Dog':
                    m = 1
                else:
                    m = 0
                class_name.append(m)
            except Valueerror as err:
                pass
    
    return img_array, class_name
            

In [4]:
#creating dataset and pre-processing the data
X_train, Y_train = create_dataset(os.path.join(path1,'Train'))
X_test, Y_test = create_dataset(os.path.join(path1,'Test'))

m_test = len(X_test)
m_train = len(X_train)

#flattening data and setting approprate dimensions  i.e , (number of data ,  height*width*3)
X_train_array = np.asarray(X_train).reshape(m_train, -1).T
X_test_array = np.asarray(X_test).reshape(m_test, -1).T
Y_train_array = np.asarray(Y_train).reshape(len(Y_train),1).T
Y_test_array = np.asarray(Y_test).reshape(len(Y_test),1).T

C:\Users\hp\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:793: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [5]:
#NOW THE MODEL PART BEGINS 
""" we have to complete that in following parts 
a. forward propagation
b. cost 
c. back propagation and update of parameters

also some additional function will be helpful...
i. sigmoid function
ii. initialization of parameters

then  we will make our model which will place this all together at one place... 
"""
def sigmoid(z):
    sig = 1 / (1 + np.exp(-z))
    return sig

In [6]:
def initialize_parameters(dim):
    #we need to initialize weigths w to zeros of certain dimension and bias term w0 = b = 0
    b = float(0)
    w = np.zeros((dim,1))
    return w,b

In [7]:
def forward_prop(X,w,b):
    A = np.dot(w.T,X) + b
    return sigmoid(A)

In [15]:
def cost(A,Y):
    m = Y.shape[1]
    #print(A.shape)
    #print(Y.shape)
    cost = np.sum(np.dot(Y,np.log(A).T) + np.dot((1-Y),np.log(1-A).T)) / (-m)
    cost = np.squeeze(np.array(cost))
    return cost

In [16]:
def back_prop(A, X, Y):
    m = Y.shape[1]
    db = np.sum(np.subtract(A,Y))/ m
    dw = np.dot(X, np.subtract(A,Y).T)/m
    return dw, db

In [17]:
def update_parameters(w,b,dw,db,learning_rate):
    w = w - learning_rate * dw
    b = b - learning_rate * db
    return w,b

In [18]:
def optimize(X,Y,w,b,num_iterations=1000,learning_rate=0.01):
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    costs = []
    for i in range(num_iterations):
        A = forward_prop(X,w,b)
        J = cost(A,Y)
        if i%100 == 0:
            print(f"cost after {i}th iteration is :{J}")
        costs.append(J)
        dw, db = back_prop(A,X,Y)
        w,b = update_parameters(w,b,dw,db,learning_rate)
    return w,b,costs
        

In [19]:
def predict(X,w,b):
    A = sigmoid(np.dot(w.T,X) + b)
    Y_predict = np.where(A > 0.5, 0, 1)
    return Y_predict

In [22]:
def model(X_train, Y_train, X_test, Y_test,num_iterations=1000,learning_rate=0.001):
    w,b = initialize_parameters(X_train.shape[0])
    w,b,costs = optimize(X_train,Y_train, w,b, num_iterations=1000,learning_rate=0.001)
    
    Y_predict_test = predict(X_test,w,b)
    Y_predict_train = predict(X_train,w,b)
    
    testa = 100 - np.mean(np.abs(Y_predict_test - Y_test)) * 100
    traina = 100 - np.mean(np.abs(Y_predict_train - Y_train)) * 100
    print(f"the test accuracy is {testa} ")
    print(f"the train accuracy is {traina}")
    
    

In [23]:
model(X_train_array, Y_train_array , X_test_array, Y_test_array)

cost after 0th iteration is :0.6931471805599464
cost after 100th iteration is :0.6905885552029964
cost after 200th iteration is :0.6890574963262823
cost after 300th iteration is :0.6880044923109803
cost after 400th iteration is :0.6872371954352133
cost after 500th iteration is :0.6866529433723969
cost after 600th iteration is :0.6861908123710463
cost after 700th iteration is :0.6858123631105563
cost after 800th iteration is :0.6854924475888415
cost after 900th iteration is :0.6852142268580647
the test accuracy is 46.909090909090914 
the train accuracy is 45.63328316908588
